In [1]:
import numpy as np
import pickle

import torch
from torch import nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

data_path = '/content/drive/MyDrive/CS7643Group/Dataset/training/parsed_data_final_srishti.pkl'
with open(data_path, "rb") as file:
  data = pickle.load(file)

print(data.shape)

data = data[:, :-1, :]
data = data[:, :, 1:]
# new_data = data[:, :, 10:]
# data = np.concatenate((another_data, new_data), axis = 2)
print(f"Shape of full dataset: {data.shape}")
# print(data[0])
X_train = data[:int(0.8*len(data)), :80, :]
X_val = data[int(0.8*len(data)): , :80, :]

y_train = data[:int(0.8*len(data)), 80:, :]
y_val = data[int(0.8*len(data)): , 80:, :]

print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

In [ ]:
import os

data_path = '/content/drive/MyDrive/CS7643Group/Dataset/parsed_data_no_timestamps.pkl'
with open(data_path, "rb") as file:
  data = pickle.load(file)
print(f"Shape of full dataset: {data.shape}")

print(data[0])

X_train = data[:int(0.8*len(data)), :80, :]
X_val = data[int(0.8*len(data)): , :80, :]

y_train = data[:int(0.8*len(data)), 80:, :]
y_val = data[int(0.8*len(data)): , 80:, :]

print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

# Initial External Actors Implementation

In [173]:
import torch
import torch.nn as nn

class TrajectoryLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_features=2, output_timesteps=10, num_layers=1):  # Set default n to 10
        super(TrajectoryLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_features * output_timesteps)
        self.output_features = output_features
        self.output_timesteps = output_timesteps

    def forward(self, x):
        # x: (batch_size, seq_length, input_size)
        _, (hn, _) = self.lstm(x)
        # hn: (num_layers, batch_size, hidden_size)
        # want to take the last layer so that you are left with (batch_size, hidden_size)
        out = self.fc(hn[-1])
        return out.view(-1, self.output_timesteps, self.output_features)  # Reshape to (batch_size, 10, input_size)

In [174]:
class TrajectoryLoss(nn.Module):
    def __init__(self):
        super(TrajectoryLoss, self).__init__()
        self.max_acceleration = 3.0

    def forward(self, predictions, targets, last_input_state, dt=0.1):
        # Reconstruct trajectory from velocities
        pred_velocity = predictions[:, :, :]
        cum_disp = torch.cumsum(pred_velocity * dt, dim=1)
        # print(pred_velocity.shape)
        # (32, 10, 2)
        pred_position = cum_disp + last_input_state.unsqueeze(1)
        # print(pred_position.shape)

        target_position = targets[:, :, :2]
        target_velocity = targets[:, :, 2:4]

        # Calculate MSE for position

        position_loss = F.mse_loss(pred_position, target_position)
        velocity_loss = F.mse_loss(pred_velocity, target_velocity)
        terminal_position_loss = F.mse_loss(pred_position[:, -1, :], target_position[:, -1, :]) + F.mse_loss(pred_position[:, 0, :], target_position[:, 0, :])
        smoothness_loss = self.smoothness_loss(pred_position)

        total_loss = position_loss + velocity_loss + smoothness_loss + 2 * terminal_position_loss
        return total_loss

    def acceleration_limit_loss(self, pred_velocity, dt=0.1):
        # Ensure accelerations remain within feasible limits by penalizing large changes in velocity
        approx_acceleration = (pred_velocity[:, :-1, :] - pred_velocity[:, 1:, :]) / dt  # Difference between consecutive velocities
        acceleration_norm = torch.norm(approx_acceleration, dim=2)
        excess_acceleration = torch.clamp(acceleration_norm - self.max_acceleration, min=0.0)
        return torch.mean(excess_acceleration ** 2)


    def smoothness_loss(self, pred_position):
        # Calculate the difference between consecutive positions
        diff = pred_position[:, :-1, :] - pred_position[:, 1:, :]
        smoothness = torch.norm(diff, dim=2)
        return torch.mean(smoothness)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import torch

train_losses = []
val_losses = []

def train_model(model, train_loader, val_loader, num_epochs=10, initial_lr=0.001, T_max=10):
    criterion = TrajectoryLoss()  # Use the custom loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr)  # Adam optimizer
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        epoch_train_loss = 0  # Initialize training loss for the epoch
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to("cuda"), batch_y.to("cuda")  # Move data to GPU

            optimizer.zero_grad()  # Zero the gradients
            outputs = model(batch_x)  # Forward pass
            loss = criterion(outputs, batch_y, batch_x[:, 0, :2])  # Compute loss using the custom loss function
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            epoch_train_loss += loss.item()  # Accumulate training loss

        # Update learning rate based on the cosine decay schedule
        scheduler.step()

        # Average training loss for the epoch
        avg_train_loss = epoch_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)  # Store training loss
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.6f}')

        # Validation loop
        model.eval()  # Set the model to evaluation mode
        epoch_val_loss = 0  # Initialize validation loss for the epoch
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to("cuda"), batch_y.to("cuda")  # Move data to GPU

                outputs = model(batch_x)
                loss = criterion(outputs, batch_y, batch_x[:, 0, :2])
                epoch_val_loss += loss.item()  # Accumulate validation loss

            # Average validation loss for the epoch
            avg_val_loss = epoch_val_loss / len(val_loader)
            val_losses.append(avg_val_loss)  # Store validation loss
            print(f'Validation Loss: {avg_val_loss:.4f}')

# Create Tensor datasets
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32))

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


# Initialize model
input_size = X_train.shape[2]  # Number of features
# hidden_size = 64  # You can adjust this

hidden_size = 32  # You can adjust this

# input_size = 5
# model = TrajectoryLSTM(input_size, hidden_size, output_features=2, output_timesteps=10, num_layers=4).to("cuda")

model = TrajectoryLSTM(input_size, hidden_size, output_features=2, output_timesteps=10, num_layers=1).to("cuda")


# Train the model with cosine decay learning rate


train_model(model, train_loader, val_loader, num_epochs=150, initial_lr=1e-2, T_max=50)

In [ ]:
x = torch.tensor(X_val, dtype=torch.float32).to("cuda")
y = torch.tensor(y_val, dtype=torch.float32).to("cuda")

with torch.no_grad():
    outputs = model(x)
    cum_disp = torch.cumsum(outputs * 0.1, dim=1)
    pred_position = cum_disp + y[:, 0, :2].unsqueeze(1)

    distances = torch.norm(pred_position - y[:, :, :2], dim=2)
    mean_ade = distances.mean().item()
#LSTM
print("Mean ADE:", mean_ade)

In [ ]:
# Choose a random example from the validation dataset
example_index = 407 # np.random.randint(0, len(X_val))
# 300 produces no movement for x and y
# 250 produces an erratic motion for x and y (curvy)

past_traj = X_val[example_index]
future_traj = y_val[example_index]

# Get the predicted future trajectory
with torch.no_grad():
  model.eval()
  X = torch.tensor(past_traj[np.newaxis, :, :], dtype=torch.float32).to("cuda")
  predicted_vels = model(X)

  predicted_vels = predicted_vels[0]
  cum_disp = torch.cumsum(predicted_vels * 0.1, dim=1)
  future_pred = cum_disp + X[0, -1, :2]
  future_pred = future_pred.cpu().numpy()



# Plot the past trajectory, actual future trajectory, and predicted future trajectory
#LSTM
plt.figure(figsize=(6, 6))
plt.plot(past_traj[:, 0], past_traj[:, 1], label='Past Trajectory', color='blue')
plt.plot(future_traj[:, 0], future_traj[:, 1], label='Actual Future Trajectory', color='red')
plt.plot(future_pred[:, 0], future_pred[:, 1], label='Predicted Future Trajectory', color='green')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title(f"Predicted vs. Actual Future Trajectory (Example {example_index})")
plt.axis('equal')
plt.legend()
plt.grid()
plt.show()

# print(past_traj[:, :2])

# Bi-directional LSTM

In [ ]:
#Kavya and Srishti

import torch
import torch.nn as nn

class TrajectoryLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_features=2, output_timesteps=10, num_layers=1):  # Set default n to 10
        super(TrajectoryLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_features * output_timesteps)
        self.output_features = output_features
        self.output_timesteps = output_timesteps

    def forward(self, x):
        # x: (batch_size, seq_length, input_size)
        # _, (hn, _) = self.lstm(x)
        # # hn: (num_layers, batch_size, hidden_size)
        # # want to take the last layer so that you are left with (batch_size, hidden_size)
        # out = self.fc(hn[-1])
        # return out.view(-1, self.output_timesteps, self.output_features)  # Reshape to (batch_size, 10, input_size)


        _, (hn, _) = self.lstm(x)
        # hn: (num_layers * num_directions, batch_size, hidden_size)
        hn = hn.view(self.lstm.num_layers, 2, x.size(0), self.lstm.hidden_size)  # Reshape to access both directions
        hn = torch.cat((hn[-1, 0], hn[-1, 1]), dim=-1)  # Concatenate forward and backward hidden states
        out = self.fc(hn)
        return out.view(-1, self.output_timesteps, self.output_features)  # Reshape to (batch_size, output_timesteps, output_features)




class TrajectoryLoss(nn.Module):
    def __init__(self):
        super(TrajectoryLoss, self).__init__()
        self.max_acceleration = 3.0

    def forward(self, predictions, targets, last_input_state, dt=0.1):
        # Reconstruct trajectory from velocities
        pred_velocity = predictions[:, :, :]
        cum_disp = torch.cumsum(pred_velocity * dt, dim=1)
        # print(pred_velocity.shape)
        # (32, 10, 2)

        pred_position = cum_disp + last_input_state.unsqueeze(1)

        # print(pred_position.shape)

        target_position = targets[:, :, :2]
        target_velocity = targets[:, :, 2:4]

        # Calculate MSE for position

        position_loss = F.mse_loss(pred_position, target_position)

        velocity_loss = F.mse_loss(pred_velocity, target_velocity)
        terminal_position_loss = F.mse_loss(pred_position[:, -1, :], target_position[:, -1, :]) + F.mse_loss(pred_position[:, 0, :], target_position[:, 0, :])
        smoothness_loss = self.smoothness_loss(pred_position)

        total_loss = position_loss + velocity_loss + smoothness_loss + 2 * terminal_position_loss
        return total_loss

    def acceleration_limit_loss(self, pred_velocity, dt=0.1):
        # Ensure accelerations remain within feasible limits by penalizing large changes in velocity
        approx_acceleration = (pred_velocity[:, :-1, :] - pred_velocity[:, 1:, :]) / dt  # Difference between consecutive velocities
        acceleration_norm = torch.norm(approx_acceleration, dim=2)
        excess_acceleration = torch.clamp(acceleration_norm - self.max_acceleration, min=0.0)
        return torch.mean(excess_acceleration ** 2)


    def smoothness_loss(self, pred_position):
        # Calculate the difference between consecutive positions
        diff = pred_position[:, :-1, :] - pred_position[:, 1:, :]
        smoothness = torch.norm(diff, dim=2)
        return torch.mean(smoothness)



from torch.utils.data import DataLoader, TensorDataset
import torch

train_losses = []
val_losses = []

def train_model(model, train_loader, val_loader, num_epochs=10, initial_lr=0.001, T_max=10):
    criterion = TrajectoryLoss()  # Use the custom loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr)  # Adam optimizer
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        epoch_train_loss = 0  # Initialize training loss for the epoch
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to("cuda"), batch_y.to("cuda")  # Move data to GPU

            optimizer.zero_grad()  # Zero the gradients
            outputs = model(batch_x)  # Forward pass
            loss = criterion(outputs, batch_y, batch_x[:, 0, :2])  # Compute loss using the custom loss function
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            epoch_train_loss += loss.item()  # Accumulate training loss

        # Update learning rate based on the cosine decay schedule
        scheduler.step()

        # Average training loss for the epoch
        avg_train_loss = epoch_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)  # Store training loss
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.6f}')

        # Validation loop
        model.eval()  # Set the model to evaluation mode
        epoch_val_loss = 0  # Initialize validation loss for the epoch
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to("cuda"), batch_y.to("cuda")  # Move data to GPU

                outputs = model(batch_x)
                loss = criterion(outputs, batch_y, batch_x[:, 0, :2])
                epoch_val_loss += loss.item()  # Accumulate validation loss

            # Average validation loss for the epoch
            avg_val_loss = epoch_val_loss / len(val_loader)
            val_losses.append(avg_val_loss)  # Store validation loss
            print(f'Validation Loss: {avg_val_loss:.4f}')

# Create Tensor datasets
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32))

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Initialize model
input_size = X_train.shape[2]  # Number of features
hidden_size = 64  # You can adjust this
# input_size = 5
model = TrajectoryLSTM(input_size, hidden_size, output_features=2, output_timesteps=10, num_layers=4).to("cuda")

# Train the model with cosine decay learning rate
train_model(model, train_loader, val_loader, num_epochs=100, initial_lr=1e-2, T_max=50)


x = torch.tensor(X_val, dtype=torch.float32).to("cuda")
y = torch.tensor(y_val, dtype=torch.float32).to("cuda")

with torch.no_grad():
    outputs = model(x)
    cum_disp = torch.cumsum(outputs * 0.1, dim=1)
    pred_position = cum_disp + y[:, 0, :2].unsqueeze(1)

    distances = torch.norm(pred_position - y[:, :, :2], dim=2)
    mean_ade = distances.mean().item()
#LSTM
print("Mean ADE:", mean_ade)

# Choose a random example from the validation dataset
example_index = 400 # np.random.randint(0, len(X_val))
past_traj = X_val[example_index]
future_traj = y_val[example_index]

# Get the predicted future trajectory
with torch.no_grad():
  model.eval()
  X = torch.tensor(past_traj[np.newaxis, :, :], dtype=torch.float32).to("cuda")
  predicted_vels = model(X)

  predicted_vels = predicted_vels[0]
  cum_disp = torch.cumsum(predicted_vels * 0.1, dim=1)
  future_pred = cum_disp + X[0, -1, :2]
  future_pred = future_pred.cpu().numpy()

# print(past_traj[:, :2])


# Plot the past trajectory, actual future trajectory, and predicted future trajectory
#LSTM
plt.figure(figsize=(6, 6))
plt.plot(past_traj[:, 0], past_traj[:, 1], label='Past Trajectory', color='blue')
plt.plot(future_traj[:, 0], future_traj[:, 1], label='Actual Future Trajectory', color='red')
plt.plot(future_pred[:, 0], future_pred[:, 1], label='Predicted Future Trajectory', color='green')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title(f"Predicted vs. Actual Future Trajectory (Example {example_index})")
plt.axis('equal')
plt.legend()
plt.grid()
plt.show()


# Additional Layers - eg. Dropout

In [ ]:
#Kavya and Srishti

import torch
import torch.nn as nn

class TrajectoryLSTM(nn.Module):
    # def __init__(self, input_size, hidden_size, output_features=2, output_timesteps=10, num_layers=1, dropout=0.2):
    #     super(TrajectoryLSTM, self).__init__()
    #     self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
    #     self.dropout = nn.Dropout(dropout)  # Dropout layer
    #     self.fc = nn.Linear(hidden_size * 2, output_features * output_timesteps)
    #     self.output_features = output_features
    #     self.output_timesteps = output_timesteps

    # def forward(self, x):
    #     # x: (batch_size, seq_length, input_size)
    #     # _, (hn, _) = self.lstm(x)
    #     # # hn: (num_layers * num_directions, batch_size, hidden_size)
    #     # hn = hn.view(self.lstm.num_layers, 2, x.size(0), self.lstm.hidden_size)  # Reshape to access both directions
    #     # hn = torch.cat((hn[-1, 0], hn[-1, 1]), dim=-1)  # Concatenate forward and backward hidden states
    #     # hn = self.dropout(hn)  # Apply dropout
    #     # out = self.fc(hn)
    #     # return out.view(-1, self.output_timesteps, self.output_features)  # Reshape to (batch_size, output_timesteps, output_features)

    #     _, (hn, _) = self.lstm(x)
    #     # hn: (num_layers, batch_size, hidden_size)
    #     # want to take the last layer so that you are left with (batch_size, hidden_size)
    #     out = self.fc(hn[-1])
    #     return out.view(-1, self.output_timesteps, self.output_features)  # Reshape to (batch_size, 10, input_size)

    def __init__(self, input_size, hidden_size, output_features=2, output_timesteps=10, num_layers=1, dropout=0.2):  # Set default n to 10
        super(TrajectoryLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout)  # Dropout layer
        # self.fc = nn.Linear(hidden_size, output_features * output_timesteps)
        self.fc = nn.Sequential(
          nn.Linear(hidden_size, 64),
          nn.BatchNorm1d(64),
          nn.Dropout(0.5), # Dropout after the first FC layer
          nn.ReLU(),
          nn.Linear(64, 32),
          nn.BatchNorm1d(32),
          nn.Dropout(0.3), # Dropout after the second FC layer
          nn.ReLU(),
          nn.Linear(32, output_features * output_timesteps))
        self.output_features = output_features
        self.output_timesteps = output_timesteps

    def forward(self, x):
        # x: (batch_size, seq_length, input_size)
        _, (hn, _) = self.lstm(x)
        # hn: (num_layers, batch_size, hidden_size)
        # want to take the last layer so that you are left with (batch_size, hidden_size)
        hn = self.dropout(hn)
        out = self.fc(hn[-1])
        return out.view(-1, self.output_timesteps, self.output_features)  # Reshape to (batch_size, 10, input_size)



class TrajectoryLoss(nn.Module):
    def __init__(self):
        super(TrajectoryLoss, self).__init__()
        self.max_acceleration = 3.0

    def forward(self, predictions, targets, last_input_state, dt=0.1):
        # Reconstruct trajectory from velocities
        pred_velocity = predictions[:, :, :]
        cum_disp = torch.cumsum(pred_velocity * dt, dim=1)
        # print(pred_velocity.shape)
        # (32, 10, 2)

        pred_position = cum_disp + last_input_state.unsqueeze(1)

        # print(pred_position.shape)

        target_position = targets[:, :, :2]
        target_velocity = targets[:, :, 2:4]

        # Calculate MSE for position

        position_loss = F.mse_loss(pred_position, target_position)

        velocity_loss = F.mse_loss(pred_velocity, target_velocity)
        terminal_position_loss = F.mse_loss(pred_position[:, -1, :], target_position[:, -1, :]) + F.mse_loss(pred_position[:, 0, :], target_position[:, 0, :])
        smoothness_loss = self.smoothness_loss(pred_position)

        total_loss = position_loss + velocity_loss + smoothness_loss + 2 * terminal_position_loss
        return total_loss

    def acceleration_limit_loss(self, pred_velocity, dt=0.1):
        # Ensure accelerations remain within feasible limits by penalizing large changes in velocity
        approx_acceleration = (pred_velocity[:, :-1, :] - pred_velocity[:, 1:, :]) / dt  # Difference between consecutive velocities
        acceleration_norm = torch.norm(approx_acceleration, dim=2)
        excess_acceleration = torch.clamp(acceleration_norm - self.max_acceleration, min=0.0)
        return torch.mean(excess_acceleration ** 2)


    def smoothness_loss(self, pred_position):
        # Calculate the difference between consecutive positions
        diff = pred_position[:, :-1, :] - pred_position[:, 1:, :]
        smoothness = torch.norm(diff, dim=2)
        return torch.mean(smoothness)



from torch.utils.data import DataLoader, TensorDataset
import torch

train_losses = []
val_losses = []

def train_model(model, train_loader, val_loader, num_epochs=10, initial_lr=0.001, T_max=10):
    criterion = TrajectoryLoss()  # Use the custom loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr)  # Adam optimizer
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        epoch_train_loss = 0  # Initialize training loss for the epoch
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to("cuda"), batch_y.to("cuda")  # Move data to GPU

            optimizer.zero_grad()  # Zero the gradients
            outputs = model(batch_x)  # Forward pass
            loss = criterion(outputs, batch_y, batch_x[:, 0, :2])  # Compute loss using the custom loss function
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            epoch_train_loss += loss.item()  # Accumulate training loss

        # Update learning rate based on the cosine decay schedule
        scheduler.step()

        # Average training loss for the epoch
        avg_train_loss = epoch_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)  # Store training loss
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.6f}')

        # Validation loop
        model.eval()  # Set the model to evaluation mode
        epoch_val_loss = 0  # Initialize validation loss for the epoch
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to("cuda"), batch_y.to("cuda")  # Move data to GPU

                outputs = model(batch_x)
                loss = criterion(outputs, batch_y, batch_x[:, 0, :2])
                epoch_val_loss += loss.item()  # Accumulate validation loss

            # Average validation loss for the epoch
            avg_val_loss = epoch_val_loss / len(val_loader)
            val_losses.append(avg_val_loss)  # Store validation loss
            print(f'Validation Loss: {avg_val_loss:.4f}')

# Create Tensor datasets
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32))

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Initialize model
input_size = X_train.shape[2]  # Number of features
hidden_size = 64  # You can adjust this
# input_size = 5
model = TrajectoryLSTM(input_size, hidden_size, output_features=2, output_timesteps=10, num_layers=1, dropout=0.5).to("cuda")

# Train the model with cosine decay learning rate
train_model(model, train_loader, val_loader, num_epochs=150, initial_lr=1e-2, T_max=50)

def plot_loss_curves(train_losses, val_losses):
    plt.figure(figsize=(8, 4))
    plt.plot(train_losses, label='Training Loss', color='blue')
    plt.plot(val_losses, label='Validation Loss', color='orange')
    plt.title('Training and Validation Loss Curves')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.show()

plot_loss_curves(train_losses, val_losses)

x = torch.tensor(X_val, dtype=torch.float32).to("cuda")
y = torch.tensor(y_val, dtype=torch.float32).to("cuda")

with torch.no_grad():
    outputs = model(x)
    cum_disp = torch.cumsum(outputs * 0.1, dim=1)
    pred_position = cum_disp + y[:, 0, :2].unsqueeze(1)

    distances = torch.norm(pred_position - y[:, :, :2], dim=2)
    mean_ade = distances.mean().item()
#LSTM
print("Mean ADE:", mean_ade)

# Choose a random example from the validation dataset
example_index = 400 # np.random.randint(0, len(X_val))
past_traj = X_val[example_index]
future_traj = y_val[example_index]

# Get the predicted future trajectory
with torch.no_grad():
  model.eval()
  X = torch.tensor(past_traj[np.newaxis, :, :], dtype=torch.float32).to("cuda")
  predicted_vels = model(X)

  predicted_vels = predicted_vels[0]
  cum_disp = torch.cumsum(predicted_vels * 0.1, dim=1)
  future_pred = cum_disp + X[0, -1, :2]
  future_pred = future_pred.cpu().numpy()

# print(past_traj[:, :2])


# Plot the past trajectory, actual future trajectory, and predicted future trajectory
#LSTM
plt.figure(figsize=(6, 6))
plt.plot(past_traj[:, 0], past_traj[:, 1], label='Past Trajectory', color='blue')
plt.plot(future_traj[:, 0], future_traj[:, 1], label='Actual Future Trajectory', color='red')
plt.plot(future_pred[:, 0], future_pred[:, 1], label='Predicted Future Trajectory', color='green')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title(f"Predicted vs. Actual Future Trajectory (Example {example_index})")
plt.axis('equal')
plt.legend()
plt.grid()
plt.show()


# Attention Mechanism

In [ ]:
#Kavya and Srishti

import torch
import torch.nn as nn

# class TrajectoryLSTM(nn.Module):
#     # def __init__(self, input_size, hidden_size, output_features=2, output_timesteps=10, num_layers=1, dropout=0.2):
#     #     super(TrajectoryLSTM, self).__init__()
#     #     self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
#     #     self.dropout = nn.Dropout(dropout)  # Dropout layer
#     #     self.fc = nn.Linear(hidden_size * 2, output_features * output_timesteps)
#     #     self.output_features = output_features
#     #     self.output_timesteps = output_timesteps

#     # def forward(self, x):
#     #     # x: (batch_size, seq_length, input_size)
#     #     # _, (hn, _) = self.lstm(x)
#     #     # # hn: (num_layers * num_directions, batch_size, hidden_size)
#     #     # hn = hn.view(self.lstm.num_layers, 2, x.size(0), self.lstm.hidden_size)  # Reshape to access both directions
#     #     # hn = torch.cat((hn[-1, 0], hn[-1, 1]), dim=-1)  # Concatenate forward and backward hidden states
#     #     # hn = self.dropout(hn)  # Apply dropout
#     #     # out = self.fc(hn)
#     #     # return out.view(-1, self.output_timesteps, self.output_features)  # Reshape to (batch_size, output_timesteps, output_features)

#     #     _, (hn, _) = self.lstm(x)
#     #     # hn: (num_layers, batch_size, hidden_size)
#     #     # want to take the last layer so that you are left with (batch_size, hidden_size)
#     #     out = self.fc(hn[-1])
#     #     return out.view(-1, self.output_timesteps, self.output_features)  # Reshape to (batch_size, 10, input_size)

#     def __init__(self, input_size, hidden_size, output_features=2, output_timesteps=10, num_layers=1, dropout=0.2):  # Set default n to 10
#         super(TrajectoryLSTM, self).__init__()
#         self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
#         self.dropout = nn.Dropout(dropout)  # Dropout layer
#         # self.fc = nn.Linear(hidden_size, output_features * output_timesteps)
#         self.fc = nn.Sequential(
#           nn.Linear(hidden_size, 64),
#           nn.BatchNorm1d(64),
#           nn.Dropout(0.5), # Dropout after the first FC layer
#           nn.ReLU(),
#           nn.Linear(64, 32),
#           nn.BatchNorm1d(32),
#           nn.Dropout(0.3), # Dropout after the second FC layer
#           nn.ReLU(),
#           nn.Linear(32, output_features * output_timesteps))
#         self.output_features = output_features
#         self.output_timesteps = output_timesteps

#     def forward(self, x):
#         # x: (batch_size, seq_length, input_size)
#         _, (hn, _) = self.lstm(x)
#         # hn: (num_layers, batch_size, hidden_size)
#         # want to take the last layer so that you are left with (batch_size, hidden_size)
#         hn = self.dropout(hn)
#         out = self.fc(hn[-1])
#         return out.view(-1, self.output_timesteps, self.output_features)  # Reshape to (batch_size, 10, input_size)



class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout=0.1):
        super(MultiHeadSelfAttention, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)

    def forward(self, x):
        # x: (batch_size, seq_length, embed_dim)
        attn_output, _ = self.attention(x, x, x)  # Self-attention mechanism
        return attn_output

class TrajectoryLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_features=2, output_timesteps=10, num_layers=1, dropout=0.5, num_heads=1):
        super(TrajectoryLSTM, self).__init__()

        # Transform input size to match attention embed_dim if needed
        self.input_transform = nn.Linear(input_size, 32)  # Transform input to embed_dim
        self.dropout = nn.Dropout(dropout)  # Dropout layer

        # MultiHeadSelfAttention layer
        self.attention = MultiHeadSelfAttention(embed_dim=64, num_heads=num_heads, dropout=dropout)

        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)

        self.layer_norm = nn.LayerNorm(hidden_size)

        # Fully connected layers for final prediction
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 64),
            nn.BatchNorm1d(64),
            nn.Dropout(0.5),  # Dropout after the first FC layer
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.Dropout(0.3),  # Dropout after the second FC layer
            nn.ReLU(),
            nn.Linear(32, output_features * output_timesteps)
        )
        self.output_features = output_features
        self.output_timesteps = output_timesteps

    def forward(self, x):
        # Transform input to match attention's expected embed_dim
        # transformed_x = self.input_transform(x)  # Transform input to embed_dim
        # transformed_x: (batch_size, seq_length, embed_dim)


        # attn_out: (batch_size, seq_length, embed_dim)

        lstm_out, _ = self.lstm(x)  # Pass attention output to LSTM
        attn_out = self.attention(lstm_out)  # Apply attention to transformed input
        # lstm_out: (batch_size, seq_length, hidden_size)

        lstm_out = self.layer_norm(lstm_out + attn_out)  # Apply LayerNorm to LSTM output
        lstm_out = lstm_out[:, -1, :]  # Take the last time step's LSTM output
        lstm_out = self.dropout(lstm_out)  # Apply dropout to LSTM output

        out = self.fc(lstm_out)  # Fully connected layer
        return out.view(-1, self.output_timesteps, self.output_features)  # Reshape to (batch_size, output_timesteps, output_features)


class TrajectoryLoss(nn.Module):
    def __init__(self):
        super(TrajectoryLoss, self).__init__()
        self.max_acceleration = 3.0

    def forward(self, predictions, targets, last_input_state, dt=0.1):
        # Reconstruct trajectory from velocities
        pred_velocity = predictions[:, :, :]
        cum_disp = torch.cumsum(pred_velocity * dt, dim=1)
        # print(pred_velocity.shape)
        # (32, 10, 2)

        pred_position = cum_disp + last_input_state.unsqueeze(1)

        # print(pred_position.shape)

        target_position = targets[:, :, :2]
        target_velocity = targets[:, :, 2:4]

        # Calculate MSE for position

        position_loss = F.mse_loss(pred_position, target_position)

        velocity_loss = F.mse_loss(pred_velocity, target_velocity)
        terminal_position_loss = F.mse_loss(pred_position[:, -1, :], target_position[:, -1, :]) + F.mse_loss(pred_position[:, 0, :], target_position[:, 0, :])
        smoothness_loss = self.smoothness_loss(pred_position)

        total_loss = position_loss + velocity_loss + smoothness_loss + 2 * terminal_position_loss
        return total_loss

    def acceleration_limit_loss(self, pred_velocity, dt=0.1):
        # Ensure accelerations remain within feasible limits by penalizing large changes in velocity
        approx_acceleration = (pred_velocity[:, :-1, :] - pred_velocity[:, 1:, :]) / dt  # Difference between consecutive velocities
        acceleration_norm = torch.norm(approx_acceleration, dim=2)
        excess_acceleration = torch.clamp(acceleration_norm - self.max_acceleration, min=0.0)
        return torch.mean(excess_acceleration ** 2)


    def smoothness_loss(self, pred_position):
        # Calculate the difference between consecutive positions
        diff = pred_position[:, :-1, :] - pred_position[:, 1:, :]
        smoothness = torch.norm(diff, dim=2)
        return torch.mean(smoothness)



from torch.utils.data import DataLoader, TensorDataset
import torch

train_losses = []
val_losses = []

def train_model(model, train_loader, val_loader, num_epochs=10, initial_lr=0.001, T_max=10, freeze_attention_epochs=20):
    criterion = TrajectoryLoss()  # Use the custom loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr)  # Adam optimizer
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    # for param in model.attention.parameters():
    #   param.requires_grad = False

    for epoch in range(num_epochs):
      # if epoch == freeze_attention_epochs:
      #   for param in model.attention.parameters():
          # param.requires_grad = True
      model.train()  # Set the model to training mode
      epoch_train_loss = 0  # Initialize training loss for the epoch
      for batch_x, batch_y in train_loader:
          batch_x, batch_y = batch_x.to("cuda"), batch_y.to("cuda")  # Move data to GPU

          optimizer.zero_grad()  # Zero the gradients
          outputs = model(batch_x)  # Forward pass
          loss = criterion(outputs, batch_y, batch_x[:, 0, :2])  # Compute loss using the custom loss function
          loss.backward()  # Backward pass
          optimizer.step()  # Update weights

          epoch_train_loss += loss.item()  # Accumulate training loss

      # Update learning rate based on the cosine decay schedule
      scheduler.step()

      # Average training loss for the epoch
      avg_train_loss = epoch_train_loss / len(train_loader)
      train_losses.append(avg_train_loss)  # Store training loss
      print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.6f}')

      # Validation loop
      model.eval()  # Set the model to evaluation mode
      epoch_val_loss = 0  # Initialize validation loss for the epoch
      with torch.no_grad():
          for batch_x, batch_y in val_loader:
              batch_x, batch_y = batch_x.to("cuda"), batch_y.to("cuda")  # Move data to GPU

              outputs = model(batch_x)
              loss = criterion(outputs, batch_y, batch_x[:, 0, :2])
              epoch_val_loss += loss.item()  # Accumulate validation loss

          # Average validation loss for the epoch
          avg_val_loss = epoch_val_loss / len(val_loader)
          val_losses.append(avg_val_loss)  # Store validation loss
          print(f'Validation Loss: {avg_val_loss:.4f}')

# Create Tensor datasets
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32))

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Initialize model
input_size = X_train.shape[2]  # Number of features
hidden_size = 64  # You can adjust this
# input_size = 5
model = TrajectoryLSTM(input_size, hidden_size, output_features=2, output_timesteps=10, num_layers=1, dropout=0.5).to("cuda")

# Train the model with cosine decay learning rate
train_model(model, train_loader, val_loader, num_epochs=150, initial_lr=1e-2, T_max=50)

def plot_loss_curves(train_losses, val_losses):
    plt.figure(figsize=(8, 4))
    plt.plot(train_losses, label='Training Loss', color='blue')
    plt.plot(val_losses, label='Validation Loss', color='orange')
    plt.title('Training and Validation Loss Curves')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.show()

plot_loss_curves(train_losses, val_losses)

x = torch.tensor(X_val, dtype=torch.float32).to("cuda")
y = torch.tensor(y_val, dtype=torch.float32).to("cuda")

with torch.no_grad():
    outputs = model(x)
    cum_disp = torch.cumsum(outputs * 0.1, dim=1)
    pred_position = cum_disp + y[:, 0, :2].unsqueeze(1)

    distances = torch.norm(pred_position - y[:, :, :2], dim=2)
    mean_ade = distances.mean().item()
#LSTM
print("Mean ADE:", mean_ade)

# Choose a random example from the validation dataset
example_index = 400 # np.random.randint(0, len(X_val))
past_traj = X_val[example_index]
future_traj = y_val[example_index]

# Get the predicted future trajectory
with torch.no_grad():
  model.eval()
  X = torch.tensor(past_traj[np.newaxis, :, :], dtype=torch.float32).to("cuda")
  predicted_vels = model(X)

  predicted_vels = predicted_vels[0]
  cum_disp = torch.cumsum(predicted_vels * 0.1, dim=1)
  future_pred = cum_disp + X[0, -1, :2]
  future_pred = future_pred.cpu().numpy()

# print(past_traj[:, :2])


# Plot the past trajectory, actual future trajectory, and predicted future trajectory
#LSTM
plt.figure(figsize=(6, 6))
plt.plot(past_traj[:, 0], past_traj[:, 1], label='Past Trajectory', color='blue')
plt.plot(future_traj[:, 0], future_traj[:, 1], label='Actual Future Trajectory', color='red')
plt.plot(future_pred[:, 0], future_pred[:, 1], label='Predicted Future Trajectory', color='green')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title(f"Predicted vs. Actual Future Trajectory (Example {example_index})")
plt.axis('equal')
plt.legend()
plt.grid()
plt.show()


In [ ]:
#Kavya and Srishti

import torch
import torch.nn as nn

class TrajectoryLSTM(nn.Module):
    # def __init__(self, input_size, hidden_size, output_features=2, output_timesteps=10, num_layers=1, dropout=0.2):
    #     super(TrajectoryLSTM, self).__init__()
    #     self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
    #     self.dropout = nn.Dropout(dropout)  # Dropout layer
    #     self.fc = nn.Linear(hidden_size * 2, output_features * output_timesteps)
    #     self.output_features = output_features
    #     self.output_timesteps = output_timesteps

    # def forward(self, x):
    #     # x: (batch_size, seq_length, input_size)
    #     # _, (hn, _) = self.lstm(x)
    #     # # hn: (num_layers * num_directions, batch_size, hidden_size)
    #     # hn = hn.view(self.lstm.num_layers, 2, x.size(0), self.lstm.hidden_size)  # Reshape to access both directions
    #     # hn = torch.cat((hn[-1, 0], hn[-1, 1]), dim=-1)  # Concatenate forward and backward hidden states
    #     # hn = self.dropout(hn)  # Apply dropout
    #     # out = self.fc(hn)
    #     # return out.view(-1, self.output_timesteps, self.output_features)  # Reshape to (batch_size, output_timesteps, output_features)

    #     _, (hn, _) = self.lstm(x)
    #     # hn: (num_layers, batch_size, hidden_size)
    #     # want to take the last layer so that you are left with (batch_size, hidden_size)
    #     out = self.fc(hn[-1])
    #     return out.view(-1, self.output_timesteps, self.output_features)  # Reshape to (batch_size, 10, input_size)

    def __init__(self, input_size, hidden_size, output_features=2, output_timesteps=10, num_layers=1, dropout=0.5):  # Set default n to 10
        super(TrajectoryLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout)  # Dropout layer
        # self.fc = nn.Linear(hidden_size, output_features * output_timesteps)
        # Single-head attention
        self.attention = nn.Linear(hidden_size, 1)  # To compute attention scores
        self.softmax = nn.Softmax(dim=1)  # For normalizing attention scores

        self.fc = nn.Sequential(
          nn.Linear(hidden_size, 64),
          nn.BatchNorm1d(64),
          nn.Dropout(0.3), # Dropout after the first FC layer
          nn.ReLU(),
          nn.Linear(64, 32),
          nn.BatchNorm1d(32),
          nn.Dropout(0.3), # Dropout after the second FC layer
          nn.ReLU(),
          nn.Linear(32, output_features * output_timesteps))
        self.output_features = output_features
        self.output_timesteps = output_timesteps

    def forward(self, x):
        # LSTM forward pass
        lstm_out, (hn, _) = self.lstm(x)
        # lstm_out: (batch_size, seq_length, hidden_size)

        # Compute attention scores
        attn_scores = self.attention(lstm_out)  # (batch_size, seq_length, 1)
        attn_weights = self.softmax(attn_scores)  # Normalize to get weights

        # Apply attention weights to LSTM outputs
        weighted_output = lstm_out * attn_weights  # (batch_size, seq_length, hidden_size)
        context_vector = torch.sum(weighted_output, dim=1)  # Sum across seq_length

        # Apply dropout and fully connected layers
        context_vector = self.dropout(context_vector)
        out = self.fc(context_vector)  # (batch_size, output_timesteps * output_features)
        return out.view(-1, self.output_timesteps, self.output_features)  # Reshape to (batch_size, output_timesteps, output_features)



class TrajectoryLoss(nn.Module):
    def __init__(self):
        super(TrajectoryLoss, self).__init__()
        self.max_acceleration = 3.0

    def forward(self, predictions, targets, last_input_state, dt=0.2):
        # Reconstruct trajectory from velocities
        pred_velocity = predictions[:, :, :]
        cum_disp = torch.cumsum(pred_velocity * dt, dim=1)
        # print(pred_velocity.shape)
        # (32, 10, 2)

        pred_position = cum_disp + last_input_state.unsqueeze(1)

        # print(pred_position.shape)

        target_position = targets[:, :, :2]
        target_velocity = targets[:, :, 2:4]

        # Calculate MSE for position

        position_loss = F.mse_loss(pred_position, target_position)

        velocity_loss = F.mse_loss(pred_velocity, target_velocity)
        terminal_position_loss = F.mse_loss(pred_position[:, -1, :], target_position[:, -1, :]) + F.mse_loss(pred_position[:, 0, :], target_position[:, 0, :])
        smoothness_loss = self.smoothness_loss(pred_position)

        # total_loss = position_loss + velocity_loss + smoothness_loss + 2 * terminal_position_loss
        total_loss = 2 * position_loss + velocity_loss + 0.5 * smoothness_loss + 3 * terminal_position_loss

        return total_loss

    def acceleration_limit_loss(self, pred_velocity, dt=0.1):
        # Ensure accelerations remain within feasible limits by penalizing large changes in velocity
        approx_acceleration = (pred_velocity[:, :-1, :] - pred_velocity[:, 1:, :]) / dt  # Difference between consecutive velocities
        acceleration_norm = torch.norm(approx_acceleration, dim=2)
        excess_acceleration = torch.clamp(acceleration_norm - self.max_acceleration, min=0.0)
        return torch.mean(excess_acceleration ** 2)


    def smoothness_loss(self, pred_position):
        # Calculate the difference between consecutive positions
        diff = pred_position[:, :-1, :] - pred_position[:, 1:, :]
        smoothness = torch.norm(diff, dim=2)
        return torch.mean(smoothness)



from torch.utils.data import DataLoader, TensorDataset
import torch

train_losses = []
val_losses = []

def train_model(model, train_loader, val_loader, num_epochs=10, initial_lr=0.001, T_max=10):
    criterion = TrajectoryLoss()  # Use the custom loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr)  # Adam optimizer
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        epoch_train_loss = 0  # Initialize training loss for the epoch
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to("cuda"), batch_y.to("cuda")  # Move data to GPU

            optimizer.zero_grad()  # Zero the gradients
            outputs = model(batch_x)  # Forward pass
            loss = criterion(outputs, batch_y, batch_x[:, 0, :2])  # Compute loss using the custom loss function
            loss.backward()  # Backward pass
            # added this to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=3)

            optimizer.step()  # Update weights

            epoch_train_loss += loss.item()  # Accumulate training loss

        # Update learning rate based on the cosine decay schedule
        scheduler.step()

        # Average training loss for the epoch
        avg_train_loss = epoch_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)  # Store training loss
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.6f}')

        # Validation loop
        model.eval()  # Set the model to evaluation mode
        epoch_val_loss = 0  # Initialize validation loss for the epoch
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to("cuda"), batch_y.to("cuda")  # Move data to GPU

                outputs = model(batch_x)
                loss = criterion(outputs, batch_y, batch_x[:, 0, :2])
                epoch_val_loss += loss.item()  # Accumulate validation loss

            # Average validation loss for the epoch
            avg_val_loss = epoch_val_loss / len(val_loader)
            val_losses.append(avg_val_loss)  # Store validation loss
            print(f'Validation Loss: {avg_val_loss:.4f}')

# Create Tensor datasets
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32))

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Initialize model
input_size = X_train.shape[2]  # Number of features
hidden_size = 128  # You can adjust this
# input_size = 5
model = TrajectoryLSTM(input_size, hidden_size, output_features=2, output_timesteps=10, num_layers=2, dropout=0.5).to("cuda")

# Train the model with cosine decay learning rate
train_model(model, train_loader, val_loader, num_epochs=150, initial_lr=1e-2, T_max=50)

def plot_loss_curves(train_losses, val_losses):
    plt.figure(figsize=(8, 4))
    plt.plot(train_losses, label='Training Loss', color='blue')
    plt.plot(val_losses, label='Validation Loss', color='orange')
    plt.title('Training and Validation Loss Curves')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.show()

plot_loss_curves(train_losses, val_losses)

x = torch.tensor(X_val, dtype=torch.float32).to("cuda")
y = torch.tensor(y_val, dtype=torch.float32).to("cuda")

with torch.no_grad():
    outputs = model(x)
    cum_disp = torch.cumsum(outputs * 0.1, dim=1)
    pred_position = cum_disp + y[:, 0, :2].unsqueeze(1)

    distances = torch.norm(pred_position - y[:, :, :2], dim=2)
    mean_ade = distances.mean().item()
#LSTM
print("Mean ADE:", mean_ade)

# Choose a random example from the validation dataset
example_index = 400 # np.random.randint(0, len(X_val))
past_traj = X_val[example_index]
future_traj = y_val[example_index]

# Get the predicted future trajectory
with torch.no_grad():
  model.eval()
  X = torch.tensor(past_traj[np.newaxis, :, :], dtype=torch.float32).to("cuda")
  predicted_vels = model(X)

  predicted_vels = predicted_vels[0]
  cum_disp = torch.cumsum(predicted_vels * 0.1, dim=1)
  future_pred = cum_disp + X[0, -1, :2]
  future_pred = future_pred.cpu().numpy()

# print(past_traj[:, :2])


# Plot the past trajectory, actual future trajectory, and predicted future trajectory
#LSTM
plt.figure(figsize=(6, 6))
plt.plot(past_traj[:, 0], past_traj[:, 1], label='Past Trajectory', color='blue')
plt.plot(future_traj[:, 0], future_traj[:, 1], label='Actual Future Trajectory', color='red')
plt.plot(future_pred[:, 0], future_pred[:, 1], label='Predicted Future Trajectory', color='green')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title(f"Predicted vs. Actual Future Trajectory (Example {example_index})")
plt.axis('equal')
plt.legend()
plt.grid()
plt.show()


# Positional Encoding

In [ ]:
#Kavya and Srishti

import math
import torch
import torch.nn as nn

class TrajectoryLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_features=2, output_timesteps=10, num_layers=1, dropout=0.5):
        super(TrajectoryLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_features = output_features
        self.output_timesteps = output_timesteps
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout)  # Dropout layer

        # Attention mechanism
        self.attention = nn.Linear(hidden_size, 1)  # To compute attention scores
        self.softmax = nn.Softmax(dim=1)  # For normalizing attention scores

        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 64),
            nn.BatchNorm1d(64),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(32, output_features * output_timesteps)
        )

    def positional_encoding(self, seq_length, dim):
        # Create the positional encoding matrix
        pos = torch.arange(seq_length, dtype=torch.float32).unsqueeze(1)
        i = torch.arange(dim, dtype=torch.float32).unsqueeze(0)

        # Compute sinusoidal functions
        angle_rates = 1 / (10000 ** (2 * (i // 2) / dim))
        angle_rads = pos * angle_rates

        # Apply sin to even indices and cos to odd indices
        pos_enc = torch.zeros((seq_length, dim))
        pos_enc[:, 0::2] = torch.sin(angle_rads[:, 0::2])  # sin for even indices
        pos_enc[:, 1::2] = torch.cos(angle_rads[:, 1::2])  # cos for odd indices

        return pos_enc

    def forward(self, x):
        batch_size, seq_length, _ = x.size()

        # Add positional encoding
        pos_enc = self.positional_encoding(seq_length, self.input_size).to(x.device)
        x = x + pos_enc.unsqueeze(0)  # Broadcast to batch size

        # LSTM forward pass
        lstm_out, (hn, _) = self.lstm(x)

        # Compute attention scores
        attn_scores = self.attention(lstm_out)  # (batch_size, seq_length, 1)
        attn_weights = self.softmax(attn_scores)  # Normalize to get weights

        # Apply attention weights to LSTM outputs
        weighted_output = lstm_out * attn_weights  # (batch_size, seq_length, hidden_size)
        context_vector = torch.sum(weighted_output, dim=1)  # Sum across seq_length

        # Apply dropout and fully connected layers
        context_vector = self.dropout(context_vector)
        out = self.fc(context_vector)  # (batch_size, output_timesteps * output_features)
        return out.view(-1, self.output_timesteps, self.output_features)  # Reshape to (batch_size, output_timesteps, output_features)


class TrajectoryLoss(nn.Module):
    def __init__(self):
        super(TrajectoryLoss, self).__init__()
        self.max_acceleration = 3.0

    def forward(self, predictions, targets, last_input_state, dt=0.2):
        # Reconstruct trajectory from velocities
        pred_velocity = predictions[:, :, :]
        cum_disp = torch.cumsum(pred_velocity * dt, dim=1)
        # print(pred_velocity.shape)
        # (32, 10, 2)

        pred_position = cum_disp + last_input_state.unsqueeze(1)

        # print(pred_position.shape)

        target_position = targets[:, :, :2]
        target_velocity = targets[:, :, 2:4]

        # Calculate MSE for position

        position_loss = F.mse_loss(pred_position, target_position)

        velocity_loss = F.mse_loss(pred_velocity, target_velocity)
        terminal_position_loss = F.mse_loss(pred_position[:, -1, :], target_position[:, -1, :]) + F.mse_loss(pred_position[:, 0, :], target_position[:, 0, :])
        smoothness_loss = self.smoothness_loss(pred_position)

        # total_loss = position_loss + velocity_loss + smoothness_loss + 2 * terminal_position_loss
        total_loss = 2 * position_loss + velocity_loss + 0.5 * smoothness_loss + 2 * terminal_position_loss

        return total_loss

    def acceleration_limit_loss(self, pred_velocity, dt=0.1):
        # Ensure accelerations remain within feasible limits by penalizing large changes in velocity
        approx_acceleration = (pred_velocity[:, :-1, :] - pred_velocity[:, 1:, :]) / dt  # Difference between consecutive velocities
        acceleration_norm = torch.norm(approx_acceleration, dim=2)
        excess_acceleration = torch.clamp(acceleration_norm - self.max_acceleration, min=0.0)
        return torch.mean(excess_acceleration ** 2)


    def smoothness_loss(self, pred_position):
        # Calculate the difference between consecutive positions
        diff = pred_position[:, :-1, :] - pred_position[:, 1:, :]
        smoothness = torch.norm(diff, dim=2)
        return torch.mean(smoothness)



from torch.utils.data import DataLoader, TensorDataset
import torch

train_losses = []
val_losses = []

def train_model(model, train_loader, val_loader, num_epochs=10, initial_lr=0.001, T_max=10):
    criterion = TrajectoryLoss()  # Use the custom loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr)  # Adam optimizer
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        epoch_train_loss = 0  # Initialize training loss for the epoch
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to("cuda"), batch_y.to("cuda")  # Move data to GPU

            optimizer.zero_grad()  # Zero the gradients
            outputs = model(batch_x)  # Forward pass
            loss = criterion(outputs, batch_y, batch_x[:, 0, :2])  # Compute loss using the custom loss function
            loss.backward()  # Backward pass
            # added this to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=3)

            optimizer.step()  # Update weights

            epoch_train_loss += loss.item()  # Accumulate training loss

        # Update learning rate based on the cosine decay schedule
        scheduler.step()

        # Average training loss for the epoch
        avg_train_loss = epoch_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)  # Store training loss
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.6f}')

        # Validation loop
        model.eval()  # Set the model to evaluation mode
        epoch_val_loss = 0  # Initialize validation loss for the epoch
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to("cuda"), batch_y.to("cuda")  # Move data to GPU

                outputs = model(batch_x)
                loss = criterion(outputs, batch_y, batch_x[:, 0, :2])
                epoch_val_loss += loss.item()  # Accumulate validation loss

            # Average validation loss for the epoch
            avg_val_loss = epoch_val_loss / len(val_loader)
            val_losses.append(avg_val_loss)  # Store validation loss
            print(f'Validation Loss: {avg_val_loss:.4f}')

# Create Tensor datasets
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32))

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Initialize model
input_size = X_train.shape[2]  # Number of features
hidden_size = 128  # You can adjust this
# input_size = 5
model = TrajectoryLSTM(input_size, hidden_size, output_features=2, output_timesteps=10, num_layers=1, dropout=0.5).to("cuda")

# Train the model with cosine decay learning rate
train_model(model, train_loader, val_loader, num_epochs=150, initial_lr=1e-2, T_max=50)

def plot_loss_curves(train_losses, val_losses):
    plt.figure(figsize=(8, 4))
    plt.plot(train_losses, label='Training Loss', color='blue')
    plt.plot(val_losses, label='Validation Loss', color='orange')
    plt.title('Training and Validation Loss Curves')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.show()

plot_loss_curves(train_losses, val_losses)

x = torch.tensor(X_val, dtype=torch.float32).to("cuda")
y = torch.tensor(y_val, dtype=torch.float32).to("cuda")

with torch.no_grad():
    outputs = model(x)
    cum_disp = torch.cumsum(outputs * 0.1, dim=1)
    pred_position = cum_disp + y[:, 0, :2].unsqueeze(1)

    distances = torch.norm(pred_position - y[:, :, :2], dim=2)
    mean_ade = distances.mean().item()
#LSTM
print("Mean ADE:", mean_ade)

# Choose a random example from the validation dataset
example_index = 400 # np.random.randint(0, len(X_val))
past_traj = X_val[example_index]
future_traj = y_val[example_index]

# Get the predicted future trajectory
with torch.no_grad():
  model.eval()
  X = torch.tensor(past_traj[np.newaxis, :, :], dtype=torch.float32).to("cuda")
  predicted_vels = model(X)

  predicted_vels = predicted_vels[0]
  cum_disp = torch.cumsum(predicted_vels * 0.1, dim=1)
  future_pred = cum_disp + X[0, -1, :2]
  future_pred = future_pred.cpu().numpy()

# print(past_traj[:, :2])


# Plot the past trajectory, actual future trajectory, and predicted future trajectory
#LSTM
plt.figure(figsize=(6, 6))
plt.plot(past_traj[:, 0], past_traj[:, 1], label='Past Trajectory', color='blue')
plt.plot(future_traj[:, 0], future_traj[:, 1], label='Actual Future Trajectory', color='red')
plt.plot(future_pred[:, 0], future_pred[:, 1], label='Predicted Future Trajectory', color='green')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title(f"Predicted vs. Actual Future Trajectory (Example {example_index})")
plt.axis('equal')
plt.legend()
plt.grid()
plt.show()


# External Actors Implementation

In [ ]:
#Kavya and Srishti
# External Actors added
import math
import torch
import torch.nn as nn

class TrajectoryLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_features=2, output_timesteps=10, num_layers=1, dropout=0.5):
        super(TrajectoryLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_features = output_features
        self.output_timesteps = output_timesteps
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout)  # Dropout layer

        # Attention mechanism
        self.attention = nn.Linear(hidden_size, 1)  # To compute attention scores
        self.softmax = nn.Softmax(dim=1)  # For normalizing attention scores

        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 64),
            nn.BatchNorm1d(64),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(32, output_features * output_timesteps)
        )

    def positional_encoding(self, seq_length, dim):
        # Create the positional encoding matrix
        pos = torch.arange(seq_length, dtype=torch.float32).unsqueeze(1)
        i = torch.arange(dim, dtype=torch.float32).unsqueeze(0)

        # Compute sinusoidal functions
        angle_rates = 1 / (10000 ** (2 * (i // 2) / dim))
        angle_rads = pos * angle_rates

        # Apply sin to even indices and cos to odd indices
        pos_enc = torch.zeros((seq_length, dim))
        pos_enc[:, 0::2] = torch.sin(angle_rads[:, 0::2])  # sin for even indices
        pos_enc[:, 1::2] = torch.cos(angle_rads[:, 1::2])  # cos for odd indices

        return pos_enc

    def forward(self, x):
        batch_size, seq_length, _ = x.size()

        # Add positional encoding
        pos_enc = self.positional_encoding(seq_length, self.input_size).to(x.device)
        x = x + pos_enc.unsqueeze(0)  # Broadcast to batch size

        # LSTM forward pass
        lstm_out, (hn, _) = self.lstm(x)

        # Compute attention scores
        attn_scores = self.attention(lstm_out)  # (batch_size, seq_length, 1)
        attn_weights = self.softmax(attn_scores)  # Normalize to get weights

        # Apply attention weights to LSTM outputs
        weighted_output = lstm_out * attn_weights  # (batch_size, seq_length, hidden_size)
        context_vector = torch.sum(weighted_output, dim=1)  # Sum across seq_length

        # Apply dropout and fully connected layers
        context_vector = self.dropout(context_vector)
        out = self.fc(context_vector)  # (batch_size, output_timesteps * output_features)
        return out.view(-1, self.output_timesteps, self.output_features)  # Reshape to (batch_size, output_timesteps, output_features)


class TrajectoryLoss(nn.Module):
    def __init__(self):
        super(TrajectoryLoss, self).__init__()
        self.max_acceleration = 3.0

    def forward(self, predictions, targets, last_input_state, dt=0.2):
        # Reconstruct trajectory from velocities
        pred_velocity = predictions[:, :, :]
        cum_disp = torch.cumsum(pred_velocity * dt, dim=1)
        # print(pred_velocity.shape)
        # (32, 10, 2)

        pred_position = cum_disp + last_input_state.unsqueeze(1)

        # print(pred_position.shape)

        target_position = targets[:, :, :2]
        target_velocity = targets[:, :, 2:4]

        # Calculate MSE for position

        position_loss = F.mse_loss(pred_position, target_position)

        velocity_loss = F.mse_loss(pred_velocity, target_velocity)
        terminal_position_loss = F.mse_loss(pred_position[:, -1, :], target_position[:, -1, :]) + F.mse_loss(pred_position[:, 0, :], target_position[:, 0, :])
        smoothness_loss = self.smoothness_loss(pred_position)

        # total_loss = position_loss + velocity_loss + smoothness_loss + 2 * terminal_position_loss
        total_loss = 2 * position_loss + velocity_loss + 0.5 * smoothness_loss + 2 * terminal_position_loss

        return total_loss

    def acceleration_limit_loss(self, pred_velocity, dt=0.1):
        # Ensure accelerations remain within feasible limits by penalizing large changes in velocity
        approx_acceleration = (pred_velocity[:, :-1, :] - pred_velocity[:, 1:, :]) / dt  # Difference between consecutive velocities
        acceleration_norm = torch.norm(approx_acceleration, dim=2)
        excess_acceleration = torch.clamp(acceleration_norm - self.max_acceleration, min=0.0)
        return torch.mean(excess_acceleration ** 2)


    def smoothness_loss(self, pred_position):
        # Calculate the difference between consecutive positions
        diff = pred_position[:, :-1, :] - pred_position[:, 1:, :]
        smoothness = torch.norm(diff, dim=2)
        return torch.mean(smoothness)



from torch.utils.data import DataLoader, TensorDataset
import torch

train_losses = []
val_losses = []

def train_model(model, train_loader, val_loader, num_epochs=10, initial_lr=0.001, T_max=10):
    criterion = TrajectoryLoss()  # Use the custom loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr)  # Adam optimizer
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        epoch_train_loss = 0  # Initialize training loss for the epoch
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to("cuda"), batch_y.to("cuda")  # Move data to GPU

            optimizer.zero_grad()  # Zero the gradients
            outputs = model(batch_x)  # Forward pass
            loss = criterion(outputs, batch_y, batch_x[:, 0, :2])  # Compute loss using the custom loss function
            loss.backward()  # Backward pass
            # added this to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=3)

            optimizer.step()  # Update weights

            epoch_train_loss += loss.item()  # Accumulate training loss

        # Update learning rate based on the cosine decay schedule
        scheduler.step()

        # Average training loss for the epoch
        avg_train_loss = epoch_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)  # Store training loss
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.6f}')

        # Validation loop
        model.eval()  # Set the model to evaluation mode
        epoch_val_loss = 0  # Initialize validation loss for the epoch
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to("cuda"), batch_y.to("cuda")  # Move data to GPU

                outputs = model(batch_x)
                loss = criterion(outputs, batch_y, batch_x[:, 0, :2])
                epoch_val_loss += loss.item()  # Accumulate validation loss

            # Average validation loss for the epoch
            avg_val_loss = epoch_val_loss / len(val_loader)
            val_losses.append(avg_val_loss)  # Store validation loss
            print(f'Validation Loss: {avg_val_loss:.4f}')

# Create Tensor datasets
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32))

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Initialize model
input_size = X_train.shape[2]  # Number of features
hidden_size = 128  # You can adjust this
# input_size = 5
model = TrajectoryLSTM(input_size, hidden_size, output_features=2, output_timesteps=10, num_layers=1, dropout=0.5).to("cuda")

# Train the model with cosine decay learning rate
train_model(model, train_loader, val_loader, num_epochs=150, initial_lr=1e-2, T_max=50)

def plot_loss_curves(train_losses, val_losses):
    plt.figure(figsize=(8, 4))
    plt.plot(train_losses, label='Training Loss', color='blue')
    plt.plot(val_losses, label='Validation Loss', color='orange')
    plt.title('Training and Validation Loss Curves')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.show()

plot_loss_curves(train_losses, val_losses)

x = torch.tensor(X_val, dtype=torch.float32).to("cuda")
y = torch.tensor(y_val, dtype=torch.float32).to("cuda")

with torch.no_grad():
    outputs = model(x)
    cum_disp = torch.cumsum(outputs * 0.1, dim=1)
    pred_position = cum_disp + y[:, 0, :2].unsqueeze(1)

    distances = torch.norm(pred_position - y[:, :, :2], dim=2)
    mean_ade = distances.mean().item()
#LSTM
print("Mean ADE:", mean_ade)

# Choose a random example from the validation dataset
example_index = 400 # np.random.randint(0, len(X_val))
past_traj = X_val[example_index]
future_traj = y_val[example_index]

# Get the predicted future trajectory
with torch.no_grad():
  model.eval()
  X = torch.tensor(past_traj[np.newaxis, :, :], dtype=torch.float32).to("cuda")
  predicted_vels = model(X)

  predicted_vels = predicted_vels[0]
  cum_disp = torch.cumsum(predicted_vels * 0.1, dim=1)
  future_pred = cum_disp + X[0, -1, :2]
  future_pred = future_pred.cpu().numpy()

# print(past_traj[:, :2])


# Plot the past trajectory, actual future trajectory, and predicted future trajectory
#LSTM
plt.figure(figsize=(6, 6))
plt.plot(past_traj[:, 0], past_traj[:, 1], label='Past Trajectory', color='blue')
plt.plot(future_traj[:, 0], future_traj[:, 1], label='Actual Future Trajectory', color='red')
plt.plot(future_pred[:, 0], future_pred[:, 1], label='Predicted Future Trajectory', color='green')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title(f"Predicted vs. Actual Future Trajectory (Example {example_index})")
plt.axis('equal')
plt.legend()
plt.grid()
plt.show()
